In [ ]:
import sys
print (sys.version)

In [ ]:
import dedupe
import unidecode

In [ ]:

from future.builtins import next
import os
import csv
import re
import collections
import logging
import optparse
from numpy import nan
import pandas as pd
import itertools

In [ ]:
import usaddress
#import probablepeople

In [ ]:
input_file = 'E:\READY\Customer table Filtered for ER\Customer_Clone_filtered_for_ER_02MAY2018.csv'
#input_file = 'customer-sample.csv'

settings_file = 'example_learned_settings_full'
training_file = 'example_training_full.json'

In [ ]:
def preProcess(column):
    import unidecode
    #column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub(' +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column:
        column = None
    return column

In [ ]:
def readData(filename):
    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        i = 0
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k,v) in row.items()]
            #row_id = int(row['Id'])
            data_d[i] = dict(clean_row)
            
            i+=1
    return data_d
print('importing data...')
data_d = readData(input_file)

In [ ]:
print(len(data_d))
print(data_d[0])
print(type(data_d))

In [ ]:
if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    fields = [
        {'field': 'CUSTOMER_NAME', 'type': 'String'},
        {'field': 'City', 'type': 'Exact', 'has missing': True},
        {'field': 'State', 'type': 'Exact', 'has missing': True},
        {'field': 'POSTAL_CODE', 'type': 'Exact', 'has missing': True},
        {'field': 'ADDRESS', 'type': 'String', 'has missing': True},        
        {'field': 'DATE_OF_BIRTH', 'type': 'Exact', 'has missing': True},
        {'field': 'SSN', 'type': 'String', 'type': 'Exact', 'has missing': True},
        {'field': 'DL', 'type': 'String', 'type': 'Exact', 'has missing': True}
    ]
    deduper = dedupe.Dedupe(fields)
    deduper.sample(data_d, 1000)
    
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.readTraining(f)
    # active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as duplicates or not.
    # use 'y', 'n', and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')
    
    dedupe.consoleLabel(deduper)
    deduper.train()
    
    with open(training_file, 'w') as tf:
        deduper.writeTraining(tf)
    
    with open(settings_file, 'wb') as sf:
        deduper.writeSettings(sf)
print('blocking...')

In [ ]:
# clustering
# find the threshold that will maximize a weighted average of our precision and recall
# when we set the recall weight to 2, we are saying we care twice as much
# about recall as we do precision
# If we had more data, we would not pass in all the blocked data into this function but 
# a representative sample.

In [ ]:
threshold = deduper.threshold(data_d, recall_weight=0.1)

In [ ]:
# match will return sets of record ISs that dedupe believes are all referring to the same entity

In [ ]:
#print(threshold)
#threshold = 0.25000045
print(threshold)

In [ ]:
print('clustering...')
clustered_dupes = deduper.match(data_d, threshold)
print('# duplicate sets', len(clustered_dupes))

In [ ]:
clustered_dupes[:10]
len(clustered_dupes)

In [ ]:
clustered_dupes[0]

In [ ]:
# write results
cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_d[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores):
        cluster_membership[record_id] = {
            "cluster id": cluster_id,
            "canonical representation": canonical_rep,
            "confidence": score
        }
singleton_id = cluster_id +1


In [ ]:
print(len(clustered_dupes))
print(cluster_id)
singleton_id = cluster_id +1
print(singleton_id)


In [ ]:
print(type(cluster_membership))
print(len(cluster_membership))
print(list(cluster_membership.keys())[1])
print(list(cluster_membership.values())[1])

In [ ]:
cluster_id

In [ ]:
output_file = 'output_full_5_10.csv'
with open(output_file, 'w',newline = '') as f_output:
    writer = csv.writer(f_output)
    
    with open(input_file) as f_input:
        reader = csv.reader(f_input)
        heading_row = next(reader)
        heading_row.insert(0, 'confidence_score')
        heading_row.insert(0, 'Cluster ID')
        #canonical_keys = canonical_rep.keys()
        #for key in canonical_keys:
        #    heading_row.append('canonical_' + key)
        writer.writerow(heading_row)
        row_id = 0
        for row in reader:         
            if row_id in cluster_membership:
                cluster_id = cluster_membership[row_id]["cluster id"]
                #canonical_rep = cluster_membership[row_id]["canonical representation"]
                row.insert(0, cluster_membership[row_id]['confidence'])
                row.insert(0, cluster_id)
                #for key in canonical_keys:
                #    row.append(canonical_rep[key].encode('utf8'))
                
            else:
                row.insert(0, None)
                row.insert(0, singleton_id)
                singleton_id += 1
                #for key in canonical_keys:
                #    row.append(None)
            row_id +=1
            writer.writerow(row)

In [ ]:
data = []
with open(input_file) as f_input:
    reader = csv.reader(f_input)
    heading_row = next(reader)
    heading_row.insert(0, 'confidence_score')
    heading_row.insert(0, 'Cluster ID')

    row_id = 0
    for row in reader:         
        if row_id in cluster_membership:
            cluster_id = cluster_membership[row_id]["cluster id"]
            row.insert(0, cluster_membership[row_id]['confidence'])
            row.insert(0, cluster_id)
            data.append(row)
                
        row_id +=1
import pandas as pd
df = pd.DataFrame(data, columns = ['Cluster ID','confidence_score','CUSTOMER_ID', 'ORGUNIT_CODE', 
                                   'FIRST_NAME', 'MIDDLE_NAMES', 'LAST_NAME', 
                                   'CUSTOMER_NAME', 'DATE_OF_BIRTH', 'ADDRESS', 'ZONE', 
                                   'POSTAL_CODE', 'PRIME_BRANCH_ID','CUSTOMER_STATUS_CODE',
                                  'City', 'State', 'SSN', 'DL'])




In [ ]:
df.to_csv('output_full_5_10_dupe_only.csv')

In [ ]:
import pickle

In [ ]:
filename = "full_threshold"
outfile = open(filename, 'wb')
pickle.dump(threshold, outfile)
outfile.close()


In [ ]:


filename = "full_clustered_dupes"
outfile = open(filename, 'wb')
pickle.dump(clustered_dupes, outfile)
outfile.close()


In [ ]:


filename = "full_cluster_membership"
outfile = open(filename, 'wb')
pickle.dump(cluster_membership, outfile)
outfile.close()
